Given the information below, find reasonable values for the prior parameters of the normal-inverse-gamma distribution — μ₀, ν₀, α₀, β₀. You will be asked to provide your values for the prior hyperparameters in class, and to explain how you came up with them.

Information provided:

- The data are normally distributed. The error margins given below represent 1 standard deviation from the mean of the parameter.

- Constraint: the mean of the data is approximately 2.3 ± 0.5.

- Constraint: the variance of the data is approximately 2.75 ± 1.

- Find μ₀, ν₀, α₀, β₀ hyperparameters for the normal-inverse-gamma prior that match this information.

In [1]:
# import relevant packages 
import numpy as np
import scipy.stats as sts
import matplotlib.pyplot as plt
import math
from scipy.optimize import minimize
from statistics import variance

In [5]:
def norminvgamma_pdf(x, sigma2, mu, nu, alpha, beta):
    """
    The pdf of the normal-inverse-gamma distribution at mean=x & variance = sigma2 
    """
    return (sts.norm.pdf(x, loc=mu, scale=np.sqrt(sigma2 / nu)) *
            sts.invgamma.pdf(sigma2, a=alpha, scale=beta))


def norminvgamma_rvs(mu, nu, alpha, beta, size=1):
    """
    This function return a matrix with (size,2) samples generated 
    from the normal-inverse-gamma distribution 
    """
    # Sample sigma^2 from the inverse-gamma distribution
    sigma2 = sts.invgamma.rvs(a=alpha, scale=beta, size=size)
    # Sample x from the normal distribution
    x = sts.norm.rvs(loc=mu, scale=np.sqrt(sigma2 / nu), size=size)
    return np.vstack((x, sigma2)).transpose()

In [53]:
def objective_function(param = init_param):
    mu, nu, alpha, beta = param[0], param[1], param[2], param[3] 
    x = mu # Mean
    sigma_2 = beta/(alpha - 1) # Variance 
    x_var = beta/((alpha - 1) * nu) # Variance of data,  for var_x taken from Wikipedia
    sigma_2_var = (beta**2) / (((alpha - 1)**2)*(alpha - 2)) # Equation for var_sigma2 taken from Wikipedia
    
    func = (x - 2.3)**2 + (sigma_2 - 2.75)**2 + (x_var - 0.5)**2 + (sigma_2_var - 1)**2

    return func 


# Let the hyperparameters be the following 
mu_0 = 2.3        # According the to prior, mean is centered around 2.3.
nu_0 = 2.1  # ν₀ accounts for the uncertainty in our prior mean: the smaller, the more uncertain 
alpha_0 = 1.5  # α₀ and β₀ account for the marginal prior over the variance.
beta_0 = 2.5

init_param = [mu_0,nu_0,alpha_0,beta_0]
output = minimize(objective_function, init_param)
updt_param = output.x

In [54]:
print(f"paramters before optimization: {init_param}")
print(f"paramters after  optimization: {updt_param}")

paramters before optimization: [2.3, 2.1, 1.5, 2.5]
paramters after  optimization: [ 2.30000059  5.50000134  9.56249617 23.54686253]


The initial hyperparameters has been updated to optimal one given the constraints on mean and variance. 